In [1]:
import torch
from torch.optim import AdamW, Adam
from torch.utils.data import DataLoader, Dataset
import os
from torch.nn.functional import one_hot

import torch
import torch.nn as nn

import joblib
import numpy as np
import math

# Data preparation

In [2]:
entokened = joblib.load("/kaggle/input/tokenized-test3/tokenized_test3.pkl")
print(len(entokened))

512


In [3]:
import random as rd
def load_and_tokenized(file):
    return file

class MyDataset(Dataset):
    def __init__(self):
        self.files = entokened

    def __len__(self):
        return len(self.files)

    def __getitem__(self, index):
        return torch.tensor(self.files[index])[:2000]

def my_collate_fn(batch):
    sequences = batch

    max_len = max([seq.size(0) for seq in sequences])

    padded_sequences = []
    padding_value = 2699

    for seq in sequences:
        padding_needed = max_len - seq.size(0)
        padded_seq = torch.nn.functional.pad(seq, (0, padding_needed), value=padding_value)
        padded_sequences.append(padded_seq)

    batch_tensor = torch.stack(padded_sequences, dim=0)
    return batch_tensor


In [4]:
dataset = MyDataset()
dataloader = DataLoader(dataset, batch_size=6, collate_fn=my_collate_fn)


In [5]:
print(dataset[10].shape)

torch.Size([1322])


# Model

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

device

'cuda'

In [7]:
class TransformerBlock(nn.Module):
    """
    A single Transformer block using Pre-LN (Layer Normalization before attention/MLP)
    and accepting boolean padding mask.
    """
    def __init__(self, embedding_dimension=128, heads=1, dropout_rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = nn.MultiheadAttention(embed_dim=embedding_dimension, num_heads=heads, dropout=dropout_rate, batch_first=True)
        self.norm1 = nn.LayerNorm(embedding_dimension)
        self.norm2 = nn.LayerNorm(embedding_dimension)
        self.MLP = nn.Sequential(
            nn.Linear(embedding_dimension, embedding_dimension * 4), 
            nn.GELU(),
            nn.Dropout(dropout_rate),
            nn.Linear(embedding_dimension * 4, embedding_dimension),
            nn.Dropout(dropout_rate)
        )
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x, key_padding_mask=None):
        seq_len = x.size(1)

        norm_x = self.norm1(x)

        causal_mask = nn.Transformer.generate_square_subsequent_mask(seq_len, device=device)
        

        attn_output, _ = self.att(norm_x, norm_x, norm_x,
                                  key_padding_mask=key_padding_mask.to(causal_mask.device),
                                  is_causal=True,
                                  need_weights=False,
                                  attn_mask=causal_mask)
        
        x = x + self.dropout(attn_output) 

        norm_x = self.norm2(x)
        mlp_output = self.MLP(norm_x)
        out = x + mlp_output 
        return out

class T1(nn.Module):
    def __init__(self, vocab_size, embedding_dimension=128, heads=1, num_transformer_blocks=1, padding_token=0, dropout_rate=0.1):
        super(T1, self).__init__()
        self.padding_token = padding_token
        self.embedding = nn.Embedding(vocab_size, embedding_dimension, padding_idx=padding_token) 
        self.pos_embedder = nn.Embedding(6000, embedding_dimension)

        self.transformer_blocks = nn.ModuleList([
            TransformerBlock(embedding_dimension=embedding_dimension, heads=heads, dropout_rate=dropout_rate)
            for _ in range(num_transformer_blocks)
        ])

        self.final_norm = nn.LayerNorm(embedding_dimension) 
        self.unembedder = nn.Linear(embedding_dimension, vocab_size)
        # self.last_act = nn.Sigmoid()


    def forward(self, input_seq):
        if input_seq.device != self.embedding.weight.device:
             input_seq = input_seq.to(self.embedding.weight.device)
             
        padding_mask = (input_seq == self.padding_token) 
        x = self.embedding(input_seq) * math.sqrt(self.embedding.embedding_dim) 
        x += self.pos_embedder(torch.arange(input_seq.shape[1]).to(x.device).unsqueeze(0))

        for block in self.transformer_blocks:
            x = block(x, key_padding_mask=padding_mask) 

        x = self.final_norm(x)

        logits = self.unembedder(x)
        #logits = self.last_act(logits)

        return logits

In [11]:
vocab_size = 2700

model = T1(vocab_size=vocab_size, embedding_dimension=250, heads=10, num_transformer_blocks=24, padding_token=2699)
model = model.to(device)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model).cuda()

optimizer = Adam(model.parameters(), lr=0.0001)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 100
total_params = sum(p.numel() for p in model.parameters())
total_params



Let's use 2 GPUs!


20931200

In [15]:
import gc
gc.collect()
torch.cuda.empty_cache()

del pred
del losses
#del model
#del optimizer
del batch


NameError: name 'pred' is not defined

In [12]:
for epoch in range(epochs):
    losses = []
    for i, batch in enumerate(dataloader):
        optimizer.zero_grad()
        batch = batch.to(device)
        preds = model(batch[:, :-1])

        batch_size, sequence_length, vocab_size = preds.shape

        loss = loss_fn(preds.view(-1, vocab_size), batch[:, 1:].contiguous().view(-1))

        loss.backward()
        optimizer.step()

        losses.append(loss.item())
        if i % 100 == 0:
             print(f"epoch: {epoch}, loss: {loss.item():.4f}")
    print(f"Epoch {epoch+1} Mean Loss: {np.mean(losses):.4f}")

/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:5849: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


epoch: 0, loss: 8.1663
Epoch 1 Mean Loss: 5.0960
epoch: 1, loss: 3.7197
Epoch 2 Mean Loss: 4.4531
epoch: 2, loss: 3.2841
Epoch 3 Mean Loss: 4.1813
epoch: 3, loss: 3.1527
Epoch 4 Mean Loss: 4.1169
epoch: 4, loss: 3.1217
Epoch 5 Mean Loss: 4.0873
epoch: 5, loss: 3.0956
Epoch 6 Mean Loss: 4.0531
epoch: 6, loss: 3.0606
Epoch 7 Mean Loss: 4.0026
epoch: 7, loss: 3.0018
Epoch 8 Mean Loss: 3.9061
epoch: 8, loss: 2.8833
Epoch 9 Mean Loss: 3.7433
epoch: 9, loss: 2.7470
Epoch 10 Mean Loss: 3.5887
epoch: 10, loss: 2.6529
Epoch 11 Mean Loss: 3.4665
epoch: 11, loss: 2.5805
Epoch 12 Mean Loss: 3.3708
epoch: 12, loss: 2.5210
Epoch 13 Mean Loss: 3.2924
epoch: 13, loss: 2.4698
Epoch 14 Mean Loss: 3.2288
epoch: 14, loss: 2.4218
Epoch 15 Mean Loss: 3.1752
epoch: 15, loss: 2.3794
Epoch 16 Mean Loss: 3.1282
epoch: 16, loss: 2.3410
Epoch 17 Mean Loss: 3.0869
epoch: 17, loss: 2.3058
Epoch 18 Mean Loss: 3.0520
epoch: 18, loss: 2.2763
Epoch 19 Mean Loss: 3.0207
epoch: 19, loss: 2.2466
Epoch 20 Mean Loss: 2.9922

KeyboardInterrupt: 

In [17]:
example = dataset[13]


In [20]:
example = torch.tensor([2691])

In [ ]:
for _ in range(60):
    with torch.no_grad():
        pred = model(example.unsqueeze(0).to(device))[0, -1, :]
        pred = torch.argmax(pred, dim=-1).cpu()
    
    example = torch.cat((example, pred.unsqueeze(0)))
example

In [21]:
from torch.distributions import Categorical

example = example.unsqueeze(0)
for _ in range(40):
    with torch.no_grad():
        logits = model(example.to(device))
        probabilities = torch.nn.functional.softmax(logits, dim=-1)
        batch_size, sequence_length, vocab_size = logits.shape
        probabilities_flat = probabilities.view(-1, vocab_size)
        dist = Categorical(probabilities_flat)
        sampled_indices_flat = dist.sample()
        pred = sampled_indices_flat.view(batch_size, sequence_length)[:, -1:]
        pred = pred.cpu()
        
    #print(example.shape, pred.shape)
    example = torch.cat((example, pred), dim=1)
example.cpu().numpy().tolist()[-100:]

[[2691,
  1549,
  1324,
  1415,
  1095,
  871,
  928,
  612,
  1640,
  1577,
  1641,
  1674,
  1546,
  1484,
  1388,
  908,
  2645,
  1747,
  1682,
  1469,
  1953,
  1730,
  2019,
  1445,
  840,
  1475,
  1731,
  1957,
  1867,
  1836,
  2632,
  1575,
  1921,
  1858,
  1730,
  1729,
  1505,
  1634,
  1635,
  1925,
  1766]]